In [2]:
import os

In [3]:
%pwd

'/Users/tien/Documents/mlProjects/textSummarization/textSummarizer/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/Users/tien/Documents/mlProjects/textSummarization/textSummarizer'

In [6]:
from dataclasses import dataclass
from pathlib import Path

# data_ingestion:
#   root_dir: artifacts/data_ingestion
#   source_URL: https://github.com/minhtien1405/textSummarizer/raw/main/summarizer-data.zip
#   local_data_file: artifacts/data_ingestion/data.zip
#   unzip_dir: artifacts/data_ingestion 
# Create a return type of the function
@dataclass(frozen=True)
class DataIngestionConfig:
  """ This is the data class, not the normal python class. """
  root_dir: Path
  source_URL: str
  local_data_file: Path
  unzip_dir: Path

In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
  def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)

    create_directories([self.config.artifacts_root])

  
  def get_data_ingestion_config(self) -> DataIngestionConfig:
    config = self.config.data_ingestion
    create_directories([config.root_dir])

    data_ingestion_config = DataIngestionConfig(
      root_dir=config.root_dir,
      source_URL=config.source_URL,
      local_data_file=config.local_data_file,
      unzip_dir=config.unzip_dir
    )

    return data_ingestion_config

In [10]:
# creating Components

import os
import urllib.request as request
import zipfile
from  textSummarizer.logging import logger
from  textSummarizer.utils.common import get_size


In [22]:
class DataIngestion:
  def __init__(self, config: DataIngestionConfig):
    self.config = config

  def download_file(self):
    if not os.path.exists(self.config.local_data_file):
      filename, headers = request.urlretrieve(
        url= self.config.source_URL,
        filename=self.config.local_data_file
      )
      logger.info(f"{filename} downloaded. With following info: \n{headers}")
    else:
      logger.info(f"File {self.config.local_data_file} already exists. {get_size(Path(self.config.local_data_file))}")


  def extract_zip_file(self):
    unzip_path = self.config.unzip_dir
    os.makedirs(unzip_path, exist_ok=True)

    with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
          zip_ref.extractall(unzip_path)


In [23]:
# Create pipeline
try:
  config = ConfigurationManager()
  data_ingestion_config = config.get_data_ingestion_config()
  data_ingestion = DataIngestion(config = data_ingestion_config)
  data_ingestion.download_file()
  data_ingestion.extract_zip_file()
except Exception as e:
  raise e

[2023-07-12 21:24:20: INFO: common: Read config/config.yaml successfully.]
[2023-07-12 21:24:20: INFO: common: Read params.yaml successfully.]
[2023-07-12 21:24:20: INFO: common: Created directory artifacts]
[2023-07-12 21:24:20: INFO: common: Created directory artifacts/data_ingestion]
[2023-07-12 21:24:22: INFO: 476732915: artifacts/data_ingestion/data.zip downloaded. With following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8B88:21DC:15B4228:1685D23:64AEFDE5
Accept-Ranges: bytes
Date: Wed, 12 Jul 2023 19:24:21 GMT
Via: 1.1 varnish
X-Served-By: cache-vie6378-VIE
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1689189861.480952,VS0,